In [1]:
import pandas as pd
import numpy as np
import pickle
import Bio   
import plotly.express as px
import plotly.graph_objects as go
from jproperties import Properties
#import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModel, TrainingArguments, Trainer, AutoConfig, DataCollatorWithPadding
from transformers import AdamW,get_scheduler
from transformers.modeling_outputs import SequenceClassifierOutput
from motif_utils import seq2kmer # Soruced from https://github.com/jerryji1993/DNABERT
import torch
from datasets import Dataset
import evaluate
import json
from load_data import create_dataset, explode_dna
import wandb
from tqdm.auto import tqdm
from transformers.onnx import FeaturesManager
import transformers
from pathlib import Path
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import torch.nn as nn
from torch.utils.data import DataLoader

c:\Users\zeusg\Envs\real-fast\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
'''
1. Normalize dataset
2. Try with smaller networks?
3. Try with very few training examples to try and get overfitting
4. Expand model size
5. Hyper Param Search https://huggingface.co/blog/ray-tune

'''

'\n1. Normalize dataset\n2. Try with smaller networks?\n3. Try with very few training examples to try and get overfitting\n4. Expand model size\n5. Hyper Param Search https://huggingface.co/blog/ray-tune\n\n'

In [3]:
#train, val, test = create_dataset()

In [4]:
with open("./data/gene_symbol_dna_sequence.pkl", 'rb') as file:
        gene =  pickle.load(file)
        
with open("./data/capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled_13022023_gene_symbol_harmonized.pkl", 'rb') as file:
        effect =  pickle.load(file)
        
        
with open("./data/gene_symbol_dna_sequence_exon.pkl", 'rb') as file:
        exon =  pickle.load(file)
        

In [5]:

#fig = go.Figure()
#fig.add_trace(go.Histogram(x = exon.Sequence.str.len()))
#fig.add_trace(go.Histogram(x = gene.Sequence.str.len()))
#fig.show()

In [6]:
def get_longest_sequence(seq_list):
    return max(seq_list, key =len)

def filter_data(effect, exon, min_seq_len = 0, max_seq_len = 512, longest_only = False):
    '''
    Filter the Data down to include only sequences within a certian size range
    Optionally includ only the genes with the longest sequences
    '''
    effect = effect[['gene_symbol','est_f_ea','p_f_ea']].copy()
    
    trimmed = exon[(exon.Sequence.str.len()>min_seq_len) & (exon.Sequence.str.len() <= max_seq_len)].copy()
    
    if longest_only:
        trimmed = trimmed.groupby(by=["Gene name"])["Sequence"].apply(list)
        trimmed = trimmed.apply(get_longest_sequence)
        trimmed = pd.DataFrame({"Gene name": trimmed.index, 'Sequence': trimmed.values})

    final = pd.merge(effect, trimmed, left_on="gene_symbol", right_on="Gene name")
    return final[["Gene name", "est_f_ea", "Sequence"]]
df = filter_data(effect, exon, 75, 512, True)
    
    

In [7]:
df.head(3)

,Gene name,est_f_ea,Sequence
0,0610009B22Rik,-0.56979,GCAGCCTTGCTCAGAGACGCATGTGCGCATGCCCGGTCGACTGAGC...
1,0610040J01Rik,-0.956256,CTATCTCTTTGATCCAGTTCAAGTGCCCTCGCCTGGTTTTGTCAAT...
2,1110017D15Rik,-0.136771,AAGAAAGCTGAGGGGGAAGATACTGAGAGGTCCCCCAAAGGAAGGG...


In [8]:
def data_prep(df, size = None):
    '''
    Preps the data for the ML pipeline
    1. Renames the columns to whats required by Hugging face
    2. Reduces the number of columns to just what's needed
    3. Randomizes the order of the data
    4. Splits into Train/val/test sets
    5. Scales the data to fit between -1,1 (Which is the range of Tanh)
    '''
    if size:
        df = df.copy().sample(frac=1)[:size]
    
    
    df_len = len(df)
    df = df.rename({"Sequence":"dna_seq", "est_f_ea":"label"}, axis=1)
    df = df[["dna_seq", "label"]]
    df.dna_seq = df.dna_seq.astype(str)
    df.dna_seq = df.dna_seq.apply(lambda x: seq2kmer(x, 6))
    df = df.sample(frac=1)
    train = df[:int(np.round(df_len*.8))].copy()
    val = df[int(np.round(df_len*.8)):int(np.round(df_len*.9))].copy()
    test = df[int(np.round(df_len*.9)):].copy()

    scaler = MinMaxScaler((-1,1))
    scaler.fit(train["label"].values.reshape(-1, 1))
    train["label"] = scaler.transform(train["label"].values.reshape(-1, 1))
    val["label"] = scaler.transform(val["label"].values.reshape(-1, 1))
    test["label"] = scaler.transform(test["label"].values.reshape(-1, 1))
    
    return train, val, test
    
train, val, test = data_prep(df)

In [9]:
'''Confirm the distributions of the 3 sets is similar'''
def show_dist():
    fig = go.Figure()
    fig.add_trace(go.Histogram(x = val["label"], histnorm='probability', name = "val"))
    fig.add_trace(go.Histogram(x = test["label"], histnorm='probability', name = "test"))
    fig.add_trace(go.Histogram(x = train["label"], histnorm='probability', name = "train"))
    
    fig.update_layout(
        barmode="overlay",
        bargap=0.1)
    fig.show()
show_dist()

In [10]:
train_mean = train.label.mean()
val_mean = val.label.mean()
test_mean = test.label.mean()

print(f"Train: {test_mean}, Val: {val_mean}, Test: {test_mean}")

Train: -0.1328170166217295, Val: -0.12846095300334928, Test: -0.1328170166217295


In [11]:
'''Tokenize the data set, and put them into dataloaders'''

tokenizer = AutoTokenizer.from_pretrained('zhihan1996/DNA_bert_6')
def tokenize_function(df):
    return tokenizer(df["dna_seq"], padding=True, truncation=True, max_length=512)#512


train = Dataset.from_pandas(train).map(tokenize_function, batched=True)
val = Dataset.from_pandas(val).map(tokenize_function, batched=True)
test = Dataset.from_pandas(test).map(tokenize_function, batched=True)

train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(train, shuffle=True, batch_size=4, collate_fn=data_collator)
val_dataloader = DataLoader(val, shuffle=True, batch_size=4, collate_fn=data_collator)


100%|██████████| 1/1 [00:00<00:00,  5.02ba/s]


In [12]:
class CustomModel(nn.Module):
    def __init__(self, checkpoint):
        super(CustomModel, self).__init__()
        self.model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(768,384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192,8)
        self.output = nn.Linear(8,1)
        self.loss_fct = nn.MSELoss()

    def forward(self, input_ids=None, attention_mask=None,labels=None):
        original_output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        outputs = self.relu(self.dropout(self.fc1(original_output[0])))
        outputs = self.relu(self.dropout(self.fc2(outputs[:,0,:])))
        outputs = self.relu(self.dropout(self.fc3(outputs)))
        logits  = self.tanh(self.output(outputs))
        
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits.squeeze(1), labels)
        wandb.log({'True Values': labels })
        wandb.log({'Predicted Values': logits.squeeze(1)})
            
        
        return SequenceClassifierOutput(loss = loss, logits=logits, hidden_states=original_output.hidden_states, attentions=original_output.attentions)
        
        
        
        
        

In [13]:
'''
Training Hyper Parameters
'''
wandb.init(project="DNA-Weight", entity="pcoady")
wandb_config = wandb.config
num_epochs = 10
log_interval = 20


model=CustomModel(checkpoint='zhihan1996/DNA_bert_6').to("cuda")
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(train)
loss_fct = nn.MSELoss()

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pcoady. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at zhihan1996/DNA_bert_6 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\zeusg\Envs\real-fast\lib\site-packages\transformers\optimization.py:306: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future versio

In [14]:
progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(val)))


dummy_labels = torch.tensor([np.array(train.data.__getitem__(1)).mean() for x in range(train.data.num_rows)])
train_baseline = loss_fct(dummy_labels, torch.tensor(np.array(train.data.__getitem__(1))))
val_baseline = loss_fct(dummy_labels[:val.data.num_rows], torch.tensor(np.array(val.data.__getitem__(1))))
    
wandb.watch(model, log_freq=100)

for epoch in range(num_epochs):
    running_loss = 0.0
    i = 0 
    model.train()
    for batch in train_dataloader: 
        batch = {k: v.to("cuda") for k, v in batch.items()}
        outputs = model(**batch)        
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        if i % log_interval ==0:
            wandb.log({f'Train Baseline':train_baseline, f'Train Loss': outputs.loss})
        
        
        progress_bar_train.update(1)
    model.eval()
    val_loss = np.array()
    for batch in val_dataloader:
        progress_bar_eval.update(1)
        

        batch = {k: v.to("cuda") for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            baseline = torch.tensor(np.array([train_mean for x in range(len(batch["labels"]))])).to("cuda")
        np.append(val_loss, np.array(outputs.loss))
    wandb.log({f'Validation Baseline':val_baseline, f'Validation Loss': val_loss})
        
        
    

  0%|          | 0/42060 [00:00<?, ?it/s]

In [ ]:
wandb.log({"Newer Test":ground_truth.cpu().numpy()})

In [ ]:
logits.squeeze(1)

In [ ]:

#model = AutoModelForSequenceClassification.from_pretrained('zhihan1996/DNA_bert_6',
#                                                           num_labels=1, 
#                                                           ignore_mismatched_sizes=True).to("cuda")
#wandb.init(project="DNA-Weight", entity="pcoady")
#wandb_config = wandb.config


def visualize_model():
        feature = "sequence-classification"
        model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature=feature)
        onnx_config = model_onnx_config(model.config)
        onnx_inputs, onnx_outputs = transformers.onnx.export(
                preprocessor=tokenizer,
                model=model,
                config=onnx_config,
                opset=13,
                output=Path("pretrained-model.onnx")
        )


In [ ]:
#evaluate.list_evaluation_modules()

In [ ]:
mse_metric = evaluate.load("mse")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    #labels = labels.reshape(-1, 1)
    baseline = np.array([val_mean for x in range(len(labels))])
    new_logits = logits.reshape(1,-1)[0]
    
    wandb.log({'Base Line': mse_metric.compute(predictions=baseline, references=labels)})
    wandb.log({'True Values': labels })
    wandb.log({'Predicted Values': new_logits})

    mse = mse_metric.compute(predictions=new_logits, references=labels)
    return mse

In [ ]:
training_args = TrainingArguments(output_dir='weight_model', 
                                  evaluation_strategy='epoch',
                                  per_device_train_batch_size = 5,  
                                  num_train_epochs=1000)

#trainer = Trainer(
#    model=model,
#    args=training_args,
#    train_dataset=train,
#    eval_dataset=train,#CHANGED
#    compute_metrics=compute_metrics
#)

In [ ]:
#trainer.train()

In [ ]:
#results = trainer.predict(train)
#fig = go.Figure()
#fig.add_trace(go.Histogram(x=results[1], histnorm='probability', name = "actual"))
#fig.add_trace(go.Histogram(x=results[0].reshape(-1, 1)[:,0], histnorm='probability', name = "predictions"))
#
#fig.update_layout(
#    barmode="overlay",
#    bargap=0.1)
#
#
#fig.show()

In [ ]:
results[0]

In [ ]:
results[1]

In [ ]:
results[0]

In [ ]:
val_mean